# Introduction

In this example we will use [Gradio](https://www.gradio.app/) to create a simple Web chatbot app. This application will be using the self hosted LLM.

## Prerequisite

Install the required modules: `langchain-community` `langchain` `gradio`

In [ ]:
!python3 -m pip install langchain-openai langchain-community langchain gradio

Define the environment parameters:

In [ ]:
from getpass import getpass

# Update the BaseURL of the model (we are using the service name)
# $ kubectl get service
base_url = "http://llm/v1"

llm_api_key  = getpass('Input the model API KEY: ')
if not llm_api_key:
    llm_api_key = "dummy"

Start the ChatBot Web application:

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
from openai import OpenAI

import gradio as gr


client = OpenAI(
  base_url = base_url,
  api_key = llm_api_key
)

#discover available models and use the first one
available_models = client.models.list()
if len(available_models.data):
    model = available_models.data[0].id
    print(f"Discovered model is: {model}")
else:
    print("No model discovered")
    sys.exit(1)


llm = ChatOpenAI(
    model=model,
    openai_api_key=llm_api_key,
    openai_api_base=base_url,
    max_tokens=256,
    temperature=0,
)

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    gpt_response = llm.invoke(history_langchain_format)
    return gpt_response.content

gr.ChatInterface(predict, title="Simple ChatBot").launch(share=True)